In [1]:
from math import ceil
import json
from os import listdir

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import ipywidgets as ipw
from IPython.display import display, clear_output
import numpy as np

from simulation import run_simulation
from graph_create import reduce_providers_simplest, powerlaw_dist_time, watts_strogatz, erdos_renyi, configuration_model
import util

In [2]:
init_notebook_mode(connected=True)

In [3]:
def plot_utils(social_utils, plot_title):
    fig = go.Figure()

    #Add averages trace
    p_vals = [ p for (p, uts) in social_utils ]
    ut_vals = [ np.average(uts) for (p, uts) in social_utils ]
    fig.add_trace(go.Scatter(x=p_vals,
        y=ut_vals, mode='lines+markers', hoverinfo='skip',
        name='avg social welfare'))
    
    #Add individual points trace
    indiv_p_vals = []
    indiv_ut_vals = []
    for (p, uts) in social_utils:
        indiv_ut_vals.extend(uts)
        indiv_p_vals.extend([p] * len(uts))
    fig.add_trace(go.Scatter(x=indiv_p_vals,
        y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='LightSkyBlue'), name='social welfare per p'))
    
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='Average social welfare (normalized)'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [4]:
# Load data
overall_data = {}
for data_file in listdir('sim_data'):
    if data_file.split('.')[1] != 'json':
        continue
    data_fp = 'sim_data/{0}'.format(data_file)
    with open(data_fp, 'r') as df:
        file_contents = json.loads(df.read())
        
        # NEEDS TO BE UPDATED AFTER NEXT DATA RUN
        for r_val, param_dict in file_contents.items():
            if r_val not in overall_data:
                overall_data[r_val] = param_dict
                continue
            for param_val, model_dict in param_dict.items():
                if param_val not in overall_data[r_val]:
                    overall_data[r_val][param_val] = model_dict
                    continue
                for model, m_data in model_dict.items():
                    if model not in overall_data[r_val][param_val]:
                        overall_data[r_val][param_val][model] = m_data

In [5]:
class ModelCompPlotter:
    def __init__(self):
        
        self.r_val_widget = ipw.FloatSlider(value=30.,
                                    min=10.,
                                    max=100.,
                                    step=10.,
                                    description='r value of simulation',
                                    disabled=False,
                                    orientation='horizontal',
                                    continuous_update=False,
                                    readout=True)
        self.param_idx_val_widget = ipw.IntSlider(value=4,
                                    min=0,
                                    max=9,
                                    step=1,
                                    description='Parameter value index',
                                    disabled=False,
                                    orientation='horizontal',
                                    continuous_update=False,
                                    readout=True)
        self.model_type_widget = ipw.RadioButtons(
                                    options=['ws', 'cm', 'er'],
                                    description='Model to show',
                                    disabled=False,
                                    orientation='horizontal',
                                    continuous_update=False,
                                    readout=True)
        
        
        #Update button
        self.plot_button = ipw.Button(
                                    description='Get plot',
                                    disabled=False,
                                    button_style='',
                                    tooltip='Get plot for simulation with given params')
        self.plot_button.on_click(self.render_visuals)

        #Widgets to render, in order
        self.render_set = [self.r_val_widget, self.param_idx_val_widget,
                           self.model_type_widget, self.plot_button]
        self.render_visuals(None)
        
    def render_visuals(self, button):
        #Reset output
        clear_output()
        
        #Render widgets
        for ritem in self.render_set:
            display(ritem)

        r_val = str(self.r_val_widget.value)
        param_vidx = self.param_idx_val_widget.value
        model = self.model_type_widget.value
        
        # CHANGE THIS FOR NEXT RUN
        param_vals = set()
        for param_dict in overall_data.values():
            model_params = [ p_val for p_val in param_dict.keys() if (model in param_dict[p_val]) ]
            param_vals = param_vals.union(set(model_params))
        param_vals = sorted(list(param_vals))
        
        param_val = param_vals[param_vidx]
        
        sim_data = overall_data[r_val][param_val][model]
        sim_utils = sim_data['utils']
        avg_diam = sim_data['avg_diam']
        
        plot_title = 'Social Utility vs p (avg diam={0})'.format(avg_diam)
        plot_utils(sim_utils, plot_title)

In [6]:
ModelCompPlotter()

FloatSlider(value=30.0, continuous_update=False, description='r value of simulation', min=10.0, step=10.0)

IntSlider(value=4, continuous_update=False, description='Parameter value index', max=9)

RadioButtons(description='Model to show', index=2, options=('ws', 'cm', 'er'), value='er')

Button(description='Get plot', style=ButtonStyle(), tooltip='Get plot for simulation with given params')